In [1]:
import numpy as np
import tensorflow as tf

<b>First pass</b>: ordinary, small convolutional networks given image data.

In [ ]:
train_img_small = np.load('datasets/train_img_64.npy')
train_lbl_small = np.load('datasets/train_lbl_64.npy')
val_img_small = np.load('datasets/val_img_64.npy')
val_lbl_small = np.load('datasets/val_lbl_64.npy')
train_img_small = np.float32(train_img_small) / 255
val_img_small = np.float32(val_img_small) / 255
print(train_img_small.shape, train_lbl_small.shape)
print(val_img_small.shape, val_lbl_small.shape)

In [ ]:
# synthetic data pre-generated by albumentations
train_img_synth = np.concatenate([np.load('datasets/train_img_64_synth.npy'), np.load('datasets/train_img_64.npy')], axis=0)
train_lbl_synth = np.concatenate([np.load('datasets/train_lbl_64_synth.npy'), np.load('datasets/train_lbl_64.npy')], axis=0)
train_img_synth = np.float32(train_img_synth) / 255
print(train_img_synth.shape, train_lbl_synth.shape)

In [19]:
dropout_prob = 0.2
spatial_dropout_prob = 0.05
reg_coef = 0.01   # experiments suggest this coef might be far too large
noise_sigma = 0.04
regulator = tf.keras.regularizers.L2(reg_coef)
this_model3 = tf.keras.Sequential([ # tf.keras.layers.Rescaling(1. / 255),
                                tf.keras.layers.GaussianNoise(noise_sigma),
                                    tf.keras.layers.Convolution2D(64, 5, activation='relu', 
                                                               padding='same', use_bias = True,
                                            input_shape = (64,64,3), kernel_regularizer=regulator),
                                 tf.keras.layers.Dropout(dropout_prob),
                                 tf.keras.layers.SpatialDropout2D(spatial_dropout_prob),
                                 tf.keras.layers.Convolution2D(64, 3, activation='relu', 
                                                               padding='same', use_bias = True,
                                                               kernel_regularizer=regulator),
                                 tf.keras.layers.MaxPool2D(strides=(2,2)), # default pool size (2,2); cuts down to 32x32xch
                                 tf.keras.layers.Convolution2D(128, 3, activation='relu', 
                                                               padding='same', use_bias = True,
                                                              kernel_regularizer=regulator),
                                 tf.keras.layers.SpatialDropout2D(spatial_dropout_prob),
                                 tf.keras.layers.Dropout(dropout_prob),
                                 tf.keras.layers.Convolution2D(128, 3, activation='relu', 
                                                               padding='same', use_bias = True,
                                                              kernel_regularizer=regulator),
                                 tf.keras.layers.MaxPool2D(strides=(2,2)), # cuts down to 16x16xch
                                 tf.keras.layers.Convolution2D(128, 3, activation='relu', 
                                                               padding='same', use_bias = True,
                                                              kernel_regularizer=regulator),
                                 tf.keras.layers.SpatialDropout2D(spatial_dropout_prob),
                                 tf.keras.layers.Dropout(dropout_prob),
                                 tf.keras.layers.Convolution2D(128, 3, activation='relu', 
                                                               padding='same', use_bias = True,
                                                              kernel_regularizer=regulator),
                                 tf.keras.layers.MaxPool2D(strides=(2,2)), # cuts down to 8x8xch
                                 tf.keras.layers.Flatten(), # 8192 outputs coming here
                                 tf.keras.layers.Dense(512, activation='relu'),
                                 tf.keras.layers.Dropout(dropout_prob),
                                 tf.keras.layers.Dense(50, activation='softmax')])
                                 
this_model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

Results: On greyscale, 0.58 val (0.81 tr) after 11 epoch. Compare 0.718 val (0.899 tr) after 19 epochs, back on colour; with dropout 0.2, reg 0.01. Regressed with dropout 0.3, reg 0.02, with 0.65 val (0.81 tr) after 17 epochs. With 64/128/256 filters, 0.72 val (0.92 tr) after 18 epochs. With 3 layers in blocks 2 and 3 (back at 64/128/128), 0.675 (0.86) after 14 epochs. With Scharr filters in input channels, 0.66 (0.87) after 14 epochs.

Moving from tanh to relu got us to 0.767 (0.95) after 19 epochs. Added GaussianNoise(0.1) and replaced Dropout with SpatialDropout(0.1). Ended at 0.658 (0.853) after 18 epochs. At this point I realised some the image set hadn't been standardised (as RGB). So I tried that again with SpatialDropout turned down to 0.05. Tried some synthetic data, things got worse. Back up some... take out all but L^2 reg, get 0.614 (0.930) after 8 epochs.

Since we still get high scores on the training set it appears the network is expressive enough (at blocks of 2, with 64/128/128 filters) to handle most of that, and getting this generalisation difference down is what we need.

So, 0.65 (0.93) after 14 epochs, with regular dropout. Next try, reintroduce gaussian noise at sigma=0.04; got 0.536 (0.88) at epoch 16. Add Dense(512) before the end; got 0.58 (0.95) at epoch 20. Return SpatialDropout, got 0.582 (0.95). Adding some synthetics, 0.608 (0.956).

In [23]:
this_model3.fit(train_img_synth, train_lbl_synth, validation_data=(val_img_small,val_lbl_small), epochs=20)

Epoch 1/20
1719/1719 [==============================] - 1192s 693ms/step - loss: 0.7291 - accuracy: 0.8630 - val_loss: 2.3349 - val_accuracy: 0.5951
Epoch 2/20
1719/1719 [==============================] - 1207s 702ms/step - loss: 0.6071 - accuracy: 0.8908 - val_loss: 2.2947 - val_accuracy: 0.6001
Epoch 3/20
1719/1719 [==============================] - 1195s 695ms/step - loss: 0.5447 - accuracy: 0.9089 - val_loss: 2.4485 - val_accuracy: 0.5882
Epoch 4/20
1719/1719 [==============================] - 1185s 689ms/step - loss: 0.4996 - accuracy: 0.9198 - val_loss: 2.7177 - val_accuracy: 0.5885
Epoch 5/20
1719/1719 [==============================] - 1179s 686ms/step - loss: 0.4688 - accuracy: 0.9274 - val_loss: 2.7426 - val_accuracy: 0.5899
Epoch 6/20
1719/1719 [==============================] - 1185s 689ms/step - loss: 0.4417 - accuracy: 0.9339 - val_loss: 2.5915 - val_accuracy: 0.6083
Epoch 7/20
1719/1719 [==============================] - 1180s 687ms/step - loss: 0.4213 - accuracy: 0.9404

KeyboardInterrupt: 

<b>Second pass</b>: The next model is a test of making a skip ("residual") connection in the network. The output of the first layer of each block becomes part of the output of the next layer. Since my blocks only have 2 layers in them this involves shrinking the output to match the size at the next block. Results were not encouraging, but I didn't try for too long.

In [19]:
class testSkipModel(tf.keras.Model):
    def __init__(self, labels, filters, rec_field, dropout_prob = 0.2, reg_coef = 0.001):
        super(testSkipModel, self).__init__()
        filters_1, filters_2, filters_3 = filters
        regulator = tf.keras.regularizers.L2(reg_coef)

        self.conv_1a = tf.keras.layers.Convolution2D(filters_1, 5, padding='same', use_bias=True, 
                                                     activation='tanh', kernel_regularizer=regulator)
        self.conv_1b = tf.keras.layers.Convolution2D(filters_1, 3, padding='same', use_bias=True, 
                                                     activation='tanh', kernel_regularizer=regulator)
    
        self.conv_2a = tf.keras.layers.Convolution2D(filters_2, 3, padding='same', use_bias=True, 
                                                     activation='tanh', kernel_regularizer=regulator)
        self.conv_2b = tf.keras.layers.Convolution2D(filters_2, 3, padding='same', use_bias=True, 
                                                     activation='tanh', kernel_regularizer=regulator)
        
        self.conv_3a = tf.keras.layers.Convolution2D(filters_3, 3, padding='same', use_bias=True, 
                                                     activation='tanh', kernel_regularizer=regulator)
        self.conv_3b = tf.keras.layers.Convolution2D(filters_3, 3, padding='same', use_bias=True, 
                                                     activation='tanh', kernel_regularizer=regulator)
        
        self.collate = tf.keras.layers.Dense(labels, kernel_regularizer=regulator, activation='softmax')
        
    def call(self, input_tensor):
        #out = tf.keras.layers.Rescaling(1. / 255)(input_tensor)
        out = tf.keras.layers.Dropout(0.2)(self.conv_1a(input_tensor))
        out_temp = tf.keras.layers.MaxPool2D(strides=(2,2))(out)
        out = tf.keras.layers.MaxPool2D(strides=(2,2))(self.conv_1b(out))
        
        out = tf.raw_ops.Concat(concat_dim=3, values=[out, out_temp]) # skip connection from 1a
        out = tf.keras.layers.Dropout(0.2)(self.conv_2a(out))
        out_temp = tf.keras.layers.MaxPool2D(strides=(2,2))(out)
        out = tf.keras.layers.MaxPool2D(strides=(2,2))(self.conv_2b(out))
        
        out = tf.raw_ops.Concat(concat_dim=3, values=[out, out_temp]) # skip connection from 2a
        out = tf.keras.layers.Dropout(0.2)(self.conv_3a(out))
        out_temp = tf.keras.layers.MaxPool2D(strides=(2,2))(out)
        out = tf.keras.layers.MaxPool2D(strides=(2,2))(self.conv_3b(out))
        
        out = tf.raw_ops.Concat(concat_dim=3, values=[out, out_temp]) # skip connection from 3a        
        out = tf.keras.layers.Flatten()(out)
        out = self.collate(out)
        return out  

In [20]:
testSkipper = testSkipModel(50, (64,128,128),3)
testSkipper.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

In [ ]:
testSkipper.fit(train_img_small, train_lbl_small, validation_data=(val_img_small, val_lbl_small), epochs=20)

<b>Third pass</b>: At this point I started experiments with the "hand geometry" output of the MediaPipe detector, which places its 21 landmarks in space. Curiously, the detector has a bit of trouble with my working dataset, only detecting a hand in about 80% of it. I do know that the detector is sensitive to colour: swapping blue/red channels will lead to non-detection. Likewise greyscale is a problem. These are not the conditions it was trained for, apparently.

Where the hand landmark data is available, it's enough alone for better results than the short CNNs I tried before. (The landmark data was previously normalised in position, orientation, and chirality.) The best score I got was 0.867 val_acc (0.94 train), with three dense layers of 256/256/256 units.

In [65]:
train_geom = np.load("datasets/train_geom.npy")
train_lbl = np.load("datasets/train_geom_lbl.npy")
val_geom = np.load("datasets/val_geom.npy")
val_lbl = np.load("datasets/val_geom_lbl.npy")
train_geom = train_geom.reshape((train_geom.shape[0],63))
val_geom = val_geom.reshape((val_geom.shape[0], 63))

In [62]:
dropout_prob = 0.2
reg_coef = 0.0001
regulator = tf.keras.regularizers.L2(reg_coef)
rng = np.random.default_rng()
layers = 4
seeds = [rng.integers(0,1024) for j in range(layers)]
inits = [tf.keras.initializers.Orthogonal(seeds[j]) for j in range(layers)]

geomModel = tf.keras.models.Sequential([#tf.keras.layers.Flatten(),
                                       tf.keras.layers.Dense(256, activation='tanh', 
                                                             #kernel_initializer = inits[0],
                                                             kernel_regularizer=regulator),
                                        #tf.keras.layers.Dropout(dropout_prob),
                                       tf.keras.layers.Dense(256, activation='tanh', 
                                                             #kernel_initializer = inits[1],
                                                             kernel_regularizer=regulator),
                                        #tf.keras.layers.Dropout(dropout_prob),
                                       tf.keras.layers.Dense(256, activation='tanh', 
                                                             #kernel_initializer = inits[2],
                                                             kernel_regularizer=regulator),
                                        #tf.keras.layers.Dropout(0.5),
                                       tf.keras.layers.Dense(50, activation='softmax', 
                                                             #kernel_initializer = inits[3],
                                                             kernel_regularizer=regulator)])

I looked into tensorflow's options for weight initialisation. Almost all of them are random initialisers, with various distributions (uniform or normal) and variances (people have looked at different normalisations in the quest to make training networks more tractable). The exception is the orthogonal initialiser, which essentially generates a random matrix like the others and then performs Gram-Schmidt/singular value decomposition on it to give an orthogonal matrix of weights.

In terms of val_acc achieved, orthogonal initialisation did not yield improvement. It did yield a puzzle: although its accuracy scores are very close to the ordinary random initialisers given like amounts of training time, the reported cross-entropy loss was much higher, by a factor of tens of thousands. (In principle there is no upper-limit to the cross-entropy, the model simply needs to give high enough confidence to a particular wrong answer.) Curious, I tried letting it run for a long time, hundreds of epochs (with the network small enough that this was a matter of minutes rather than days). The cross-entropy does eventually come down, but the accuracy does nothing special. This sort of behaviour makes me think there must be interesting things to say about (for lack of a better expression) the dynamics of NN learning, but I don't know what they might be.

In [63]:
learning_rate=0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                    learning_rate,
                    decay_steps=20000,
                    decay_rate=0.9,
                    staircase=True)

geomModel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

In [72]:
geomModel.fit(train_geom, train_lbl, validation_data=(val_geom, val_lbl), epochs=10)

Epoch 1/10
699/699 [==============================] - 3s 5ms/step - loss: 51351.5000 - accuracy: 0.8723 - val_loss: 51276.8594 - val_accuracy: 0.7334
Epoch 2/10
699/699 [==============================] - 3s 5ms/step - loss: 51201.8164 - accuracy: 0.8730 - val_loss: 51127.4414 - val_accuracy: 0.7340
Epoch 3/10
699/699 [==============================] - 3s 4ms/step - loss: 51052.5977 - accuracy: 0.8752 - val_loss: 50978.5000 - val_accuracy: 0.7325
Epoch 4/10
699/699 [==============================] - 3s 4ms/step - loss: 50903.8633 - accuracy: 0.8795 - val_loss: 50829.9180 - val_accuracy: 0.7329
Epoch 5/10
699/699 [==============================] - 3s 4ms/step - loss: 50755.3398 - accuracy: 0.8802 - val_loss: 50681.5039 - val_accuracy: 0.7358
Epoch 6/10
699/699 [==============================] - 3s 4ms/step - loss: 50607.0312 - accuracy: 0.8841 - val_loss: 50533.2617 - val_accuracy: 0.7397
Epoch 7/10
699/699 [==============================] - 3s 5ms/step - loss: 50458.9336 - accuracy: 0.8

In [ ]:
epochs = 200
for j in range(0, epochs):
    geomModel.fit(train_geom, train_lbl, epochs=1)
    geomModel.evaluate(val_geom, val_lbl, verbose=2)
# 200 epochs later... "you haven't converged or blown up yet? another round! Adam, what a dogged searcher."

<b>Fourth pass</b>: models combining image and geometric data. I'm looking at an attention-type mechanism where a short network uses the geometry to make weights for the convolutional network. Since the geometric data isn't there for every frame it also tries to train a 'back-up' layer just from the image data. It works better than previous tries. There's still a lot I don't know.

In [316]:
train_geom_full = np.load("datasets/train_geom_full.npy")
val_geom_full = np.load("datasets/val_geom_full.npy")
print(train_geom_full.shape, val_geom_full.shape)

(26573, 21, 3) (5958, 21, 3)


In [317]:
train_geom_full = train_geom_full.reshape((train_geom_full.shape[0], 63))
val_geom_full = val_geom_full.reshape((val_geom_full.shape[0],63))

In [318]:
train_img_small = np.load('datasets/train_img_64.npy')
train_lbl_small = np.load('datasets/train_lbl_64.npy')
val_img_small = np.load('datasets/val_img_64.npy')
val_lbl_small = np.load('datasets/val_lbl_64.npy')
print(train_img_small.shape, val_img_small.shape)
print(train_lbl_small.shape, val_lbl_small.shape)
train_img_small = np.float32(train_img_small) / 255
val_img_small = np.float32(val_img_small) / 255

(26573, 64, 64, 3) (5958, 64, 64, 3)
(26573,) (5958,)


Tensorboard is a profiling add-on. it can tell you lots of things about the statistics of your model's weights,
how much time it takes doing what operations, and a lot more. I've barely taken a look.

https://www.tensorflow.org/tensorboard

In [58]:
%load_ext tensorboard

In [59]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = '2000,2010')
# the data it logs can take up a lot of space, so they recommend using it only for 10 or 20 steps to gather its statistics,
# and not steps at the beginning, where there can be overhead etc.

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
# Regularisation stuff...
# tf.keras.layers.GaussianNoise(noise_sigma) (last used sigma = 0.04)
# tf.keras.regularizers.L2(reg_coef) (last used coef 0.001 or 0.0001?)
# tf.keras.layers.Dropout(dropout_prob) (last used prob = 0.2)
# tf.keras.layers.SpatialDropout2D(spatial_dropout_prob) (last used prob = 0.05)

# Augmentation stuff -- when using gpu it's advised to stick this on the dataset; as long as the preprocessing
# consists only of tensorflow Graph-able operations it'll be executed in parallel when data is about to be called from it

# train_img_tf = tf.data.Dataset.from_tensor_slices(train_img_small)
# train_img_tf.map(lambda x: pre_process(x)), where pre_process could be a keras.Sequential object

# tf.keras.layers.RandomBrightness(factor, value_range=(0, 1)) (factor = pair of floats in [-1,1])
# tf.keras.layers.RandomContrast(factor in [0,1])
# tf.keras.layers.RandomFlip(mode='horizontal')
# tf.keras.layers.RandomRotation(fill_mode='constant', factor in [0,1]), rotation up to angle factor*2pi
# tf.keras.layers.RandomZoom(height_factor=0.2, fill_mode='constant')  default arg width_factor=None preserves aspect ratio

In [319]:
synthesiser_train = tf.keras.Sequential([tf.keras.layers.RandomBrightness(0.15, value_range=(0,1)),
                                    tf.keras.layers.RandomFlip(mode = 'horizontal'),
                                    tf.keras.layers.RandomRotation(0.04, fill_mode='constant'),
                                    tf.keras.layers.RandomZoom(height_factor=0.2, fill_mode='constant')])
batch_size = train_img_small.shape[0]
train_img_tf = tf.data.Dataset.from_tensor_slices(train_img_small)
train_geom_tf = tf.data.Dataset.from_tensor_slices(train_geom_full)

# this makes a dataset object with an attached function, rather than just applying a function once to its tensors
train_synth = train_img_tf.map(lambda x: synthesiser_train(x),
                                 num_parallel_calls=batch_size).batch(batch_size)
train_proc = train_synth.get_single_element()

In [320]:
# why two copies of the same object? because tensorflow handles batch size in a way I don't understand, and
# using the same one in two places raises errors
synthesiser_val = tf.keras.Sequential([tf.keras.layers.RandomBrightness(0.15, value_range=(0,1)),
                                    tf.keras.layers.RandomFlip(mode = 'horizontal'),
                                    tf.keras.layers.RandomRotation(0.04, fill_mode='constant'),
                                    tf.keras.layers.RandomZoom(height_factor=0.2, fill_mode='constant')])

In [321]:
class testAttentionModel(tf.keras.Model):
    def __init__(self, conv_filters, reg_coef=0, labels=50):
        super(testAttentionModel, self).__init__()
        filters_1, filters_2, filters_3 = conv_filters
        self.reg = tf.keras.regularizers.L2(reg_coef)
        conv_out_size = 8*8*filters_3
        self.spatial_dropout_prob = 0.02
        self.dropout_prob = 0.1
        
        # 64x64xch
        self.conv_1a = tf.keras.layers.Convolution2D(filters_1, 5, padding='same', use_bias=True, 
                                                     activation='relu',
                                                     kernel_regularizer=self.reg)
        self.conv_1b = tf.keras.layers.Convolution2D(filters_1, 3, padding='same', use_bias=True, 
                                                     activation='relu',
                                                     kernel_regularizer=self.reg)
        # 32x32xch
        self.conv_2a = tf.keras.layers.Convolution2D(filters_2, 3, padding='same', use_bias=True, 
                                                     activation='relu',
                                                     kernel_regularizer=self.reg)
        self.conv_2b = tf.keras.layers.Convolution2D(filters_2, 3, padding='same', use_bias=True,
                                                     activation='relu',
                                                     kernel_regularizer=self.reg)
        # 16x16xch
        self.conv_3a = tf.keras.layers.Convolution2D(filters_3, 3, padding='same', use_bias=True, 
                                                     activation='relu',
                                                    kernel_regularizer=self.reg)
        self.conv_3b = tf.keras.layers.Convolution2D(filters_3, 3, padding='same', use_bias=True, 
                                                     activation='relu',
                                                     kernel_regularizer=self.reg)
        # 8x8xch
        #self.conv_4a = tf.keras.layers.Convolution2D(filters_4, 3, padding='same', use_bias=True, activation='relu')
                                                    # activation='tanh', kernel_regularizer=regulator)
        #self.conv_4b = tf.keras.layers.Convolution2D(filters_4, 3, padding='same', use_bias=True, activation='relu')
                                                     #activation='tanh', kernel_regularizer=regulator)
        # out: 4x4xch
        
        self.geom1 = tf.keras.layers.Dense(64, use_bias=True, activation='relu', kernel_regularizer=self.reg)
        self.geom_backup = tf.keras.layers.Dense(64, use_bias=True, activation='relu')
        self.geom2 = tf.keras.layers.Dense(conv_out_size, use_bias=True, activation='relu', kernel_regularizer=self.reg)
        #self.dense3 = tf.keras.layers.Dense(units_3, use_bias=True, 
        #                                             activation='tanh', kernel_regularizer=regulator)
        self.policy = tf.keras.layers.Dense(labels, activation='softmax')
        
    def call(self, input_list, training=True):
        #input_layer = tf.reshape(input_list[0], [-1, 64, 64, 3])
        c_out = tf.keras.layers.GaussianNoise(0.03)(input_list[0], training=training)
        c_out = tf.keras.layers.SpatialDropout2D(self.spatial_dropout_prob)(self.conv_1a(c_out),
                                                                            training=training)
        c_out = tf.keras.layers.MaxPool2D(strides=(2,2))(self.conv_1b(c_out))
        c_out = tf.keras.layers.SpatialDropout2D(self.spatial_dropout_prob)(self.conv_2a(c_out),
                                                                            training=training)
        c_out = tf.keras.layers.MaxPool2D(strides=(2,2))(self.conv_2b(c_out))
        c_out = tf.keras.layers.SpatialDropout2D(self.spatial_dropout_prob) (self.conv_3a(c_out),
                                                                            training=training)
        c_out = tf.keras.layers.MaxPool2D(strides=(2,2))(self.conv_3b(c_out))
        #c_out = self.conv_4a(c_out)
        #c_out = tf.keras.layers.MaxPool2D(strides=(2,2))(self.conv_4b(c_out))
        c_output = tf.keras.layers.Flatten()(c_out)
       
        g_out = self.geom1(input_list[1])
        if tf.math.reduce_max(g_out) == 0:
            g_out = self.geom_backup(tf.keras.layers.Flatten()(input_list[0]))
        g_out = tf.keras.layers.Dropout(self.dropout_prob)(g_out, training=training)
        g_out = tf.keras.layers.Dropout(self.dropout_prob)(self.geom2(g_out),training=training)
       
        return self.policy(tf.keras.layers.Multiply()([c_output, g_out]))

In [322]:
testAttender2 = testAttentionModel((64,128,256), reg_coef=0.001)

In [323]:
learning_rate=0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                    learning_rate,
                    decay_steps=5000,
                    decay_rate=0.9,
                    staircase=True)

testAttender2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

Results: 0.790 (0.964) with 3 blocks 64/128/256. Reg at 0.001 didn't help, 0.77 (0.966). (At this point cut half of the dataset grass.) Added spatial/regular dropout at 0.04/0.2, reg=0.001. Slower, val stalled around .745 (tr continued up to .93). adding in synth data, tr_acc (on the same model) went down to .745 too. but while it recovered val did not.

In [ ]:
testAttender2.fit([train_proc, train_geom_full], train_lbl_small,
                 validation_data=([val_img_small, val_geom_full], val_lbl_small),
                 epochs=80)

Epoch 1/80
831/831 [==============================] - 643s 769ms/step - loss: 2.1825 - accuracy: 0.5577 - val_loss: 1.9516 - val_accuracy: 0.6015
Epoch 2/80
831/831 [==============================] - 640s 770ms/step - loss: 1.6063 - accuracy: 0.6559 - val_loss: 1.8304 - val_accuracy: 0.6170
Epoch 3/80
831/831 [==============================] - 639s 769ms/step - loss: 1.4515 - accuracy: 0.6765 - val_loss: 1.7214 - val_accuracy: 0.6299
Epoch 4/80
831/831 [==============================] - 651s 783ms/step - loss: 1.3320 - accuracy: 0.7010 - val_loss: 1.6108 - val_accuracy: 0.6482
Epoch 5/80
831/831 [==============================] - 650s 782ms/step - loss: 1.2167 - accuracy: 0.7218 - val_loss: 1.4926 - val_accuracy: 0.6804
Epoch 6/80
831/831 [==============================] - 642s 773ms/step - loss: 1.1065 - accuracy: 0.7464 - val_loss: 1.4383 - val_accuracy: 0.6841
Epoch 7/80
831/831 [==============================] - 632s 761ms/step - loss: 1.0190 - accuracy: 0.7674 - val_loss: 1.3759 -

In [ ]:
# this model tries out test-time data augmentation; that is, given an image it generates some random synthetic frames 
# from it, gives those to the underlying trained model, and returns their averaged probabilities
class testPollModel(tf.keras.Model):
    def __init__(self, polled_model, size):
        super(testPollModel, self).__init__()
        self.size = size
        self.polled_model = polled_model
        
    def call(self, input_list):
        vote_list = []
        for j in range(self.size):
            vote_list.append(self.polled_model(input_list = [synthesiser_val(input_list[0]), input_list[1]], 
                                               training=False))
        return tf.keras.layers.Average()(vote_list)
# I'm uncertain whether this is computationally the most efficient route. tensorflow does a lot of automatic optimisation
# (I've learned) but there's something here it really doesn't like.